# Lindenmayer Systems

This notebook demonstrates a variety of fractals that you can generate using Lindenmayer systems, or L-systems. They were developed by a biologist, Aristid Lindenmayer, who was interested in modeling the development of plant cells.

L-systems use relatively simple string substitution rules as recipes to generate fractals -- geometric forms that "grow" by creating copies of of themselves. Some of these rules can be expressed in just a few characters, but generate remarkably complex, life-like forms.

This notebook was inspired by [LSystemBot 2.0](https://twitter.com/lsystembot?lang=en), and many of the fractals it generates were originally randomly generated by its engine.

This is a work in progress, but if you run all the cells until you reach the bottom, you'll find a dropdown that allows you to select and display one of a multitude of L-system fractals.

In [ ]:
from math import pi, sin, cos
from collections.abc import Sequence
from pprint import pprint
import random

import ipywidgets as widgets
from IPython.display import display

import rules

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from matplotlib import pyplot as plt

In [ ]:
class Lsystem(Sequence):
    """A class representing an abstract sequence of lsystem iterations."""
    def __init__(self, start, rules, max_iters=5, angle=90, bindings={}):
        self.start = start
        self.rules = rules
        self.max_iters = max_iters
        self._cache = []
        self._bindings = {}
        self.set_default_bindings(angle)
        if bindings:
            self.set_bindings(bindings)

    @classmethod
    def from_lsbot(cls, start, rules, a=90, iter=5, wiggly=None, bindings={}):
        """Designed to accept the lsystem representation used by the LSystem Bot,
           which is why it masks the `iter` built-in."""
        return cls(start, rules, iter, a, bindings=bindings)
        
    def set_default_bindings(self, angle=90):
        self.bind_push('[')
        self.bind_pop(']')
        self.bind_turn('+', angle)
        self.bind_turn('-', -angle)
        self.bind_step('F', 1)
        
    def set_bindings(self, bindings):
        self._bindings.update(bindings)
    
    def get_bindings(self):
        return self._bindings
    
    def bind_push(self, char):
        self._bind('push', char)

    def bind_pop(self, char):
        self._bind('pop', char) 
        
    def bind_turn(self, char, arg):
        self._bind('turn', char, arg)
    
    def bind_step(self, char, arg):
        self._bind('step', char, arg)
    
    def _bind(self, action, char, arg=None):
        self._bindings[char] = (action, arg)
        
    def interpret(self, ix):
        state = self[ix]
        position = (0, 0)
        heading = 0
        stack = []
        points = [[position]]
        for c in state:
            if c not in self._bindings:
                continue
            method, arg = self._bindings[c]
            if method == 'push':
                stack.append((position, heading))
            elif method == 'pop':
                position, heading = stack.pop()
                points.append([position])
            elif method == 'turn':
                heading += arg
            elif method == 'step':
                x, y = position
                x += arg * cos(pi * heading / 180)
                y += arg * sin(pi * heading / 180)
                position = x, y
                points[-1].append(position)

        return points

    @staticmethod
    def lsystem_step(state, rules):
        lstring_subs = [rules[c] if c in rules else c
                        for c in state]
        return ''.join(lstring_subs)

    def __len__(self):
        return self.max_iters

    def __getitem__(self, ix):
        if -self.max_iters <= ix < 0:
            ix = self.max_iters - ix

        last = len(self._cache)
        if last <= ix < self.max_iters:
            state = self._cache[-1] if self._cache else self.start
            for n in range(last, ix + 1):
                state = self.lsystem_step(state, self.rules)
                self._cache.append(state)

        return self._cache[ix]

def plot_series(series, 
                colormap='plasma',  # 'viridis' also good
                linewidth=0.25,
                clearfig=True,
                savefig=False):
    if clearfig:
        plt.clf()
    
    plt.axes().set_aspect('equal')
    
    cmap = plt.get_cmap(colormap)
    colors = [cmap(i * 1.0 / len(series)) for i in range(len(series))]
    for i, (x, y) in enumerate(series):
        plt.plot(x, y, linewidth=linewidth, color=colors[i])

    if savefig:
        if isinstance(savefig, basestring):
            plt.savefig('{}.pdf'.format(savefig))
        else:
            plt.savefig('untitled.pdf')
    else:
        plt.show()

In [ ]:
rule_list = [r for r in dir(rules) if not r.startswith('_')]
rule_dropdown = widgets.Dropdown(
    options=rule_list,
    value='hexaflower',
    description='Rule: ',
    disabled=False,
)

def render_rule(args):
    rulename = args['new']
    rule = getattr(rules, rulename).copy()
    
    if rulename == 'randrules':
        rule = random.choice(rule)

    if rule['iter'] > 8:
        rule['iter'] = 8

    lsystem = Lsystem.from_lsbot(**rule)
    series = lsystem.interpret(rule['iter'] - 1)
    series = [list(zip(*s)) for s in series]
    
    pprint(rule)
    plot_series(series, linewidth=0.5)

    
rule_dropdown.observe(render_rule, names='value')
display(rule_dropdown)
render_rule({'new': 'hexaflower'})

In [ ]:
# Dump all the figures from randrules into stand-alone files for review:

# for i, r in enumerate(randrules):
#    r['iter'] = r.get('iter', 5)
#    iters = r['iter'] - 1
#    lsystem = Lsystem.from_lsbot(**r)
#    series = lsystem.interpret(iters)
#    series = [zip(*s) for s in series]
#    plot_series(series, linewidth=0.25, savefig='temp_{}'.format(i))